### preprocessing data dan modelling

In [ ]:
import pandas as pd
from scipy.stats import linregress

# Read data dari file excel
df = pd.read_excel('data 24 may.xlsx', sheet_name=['75', '70', '65', '60', '55', '50', '45', '40', '35', '30'])

# Buat headers
headers = ['SimpleRx Starting', 'Time', 'Module', 'Temperature', 'Humidity']

# Looping untuk tiap sheet (untuk variasi jarak)
for sheet_name in df.keys():
    df[sheet_name].columns = headers
    
for sheet_name in df.keys():
    # Convert kolom 'Time' menjadi tipe string
    if not isinstance(df[sheet_name]['Time'].iloc[0], str):
        df[sheet_name]['Time'] = df[sheet_name]['Time'].astype(str)

    # Replace '.' dengan ':' pada kolom 'Time' 
    df[sheet_name]['Time'] = df[sheet_name]['Time'].str.replace('.', ':', regex=False)

    # Konversi waktu ke detik 'TimeInSeconds'
    df[sheet_name]['TimeInSeconds'] = pd.to_timedelta(df[sheet_name]['Time']).dt.total_seconds()
    df[sheet_name]['TimeInSeconds'] -= df[sheet_name]['TimeInSeconds'].min() - 1

    # Hapus duplicate rows berdasarkan semua kolom
    df[sheet_name].drop_duplicates(inplace=True)

print(df)

In [ ]:
import matplotlib.pyplot as plt

# Buat plot temperatur terhadap waktu
for sheet_name in df.keys():
    plt.plot(df[sheet_name]['TimeInSeconds'], df[sheet_name]['Temperature'], label=sheet_name +' cm')

plt.xlim(0, 300) 
plt.title("Plot Temperatur terhadap Waktu")
plt.ylabel("Temperatur (°C)")
plt.xlabel("Waktu (s)")
plt.legend()
plt.show()

In [ ]:
# buat dataset yang berisi gradien dan jarak 
slopes = []
for sheet_name in df.keys():
    data = df[sheet_name]
    slope, _, _, _, _ = linregress(data['TimeInSeconds'], data['Temperature'])
    slopes_dict = {'Jarak (cm)': sheet_name, 'Gradien': slope}
    slopes.append(slopes_dict)
    
df_slopes = pd.DataFrame(slopes)
df_slopes

In [ ]:
# Convert tipe data ke integer
df_slopes['Jarak (cm)'] = df_slopes['Jarak (cm)'].astype(int)
print(df_slopes.dtypes)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Fitting regresi linear untuk gradien terhadap jarak 
slope, intercept, r_value, p_value, std_err = linregress(df_slopes['Jarak (cm)'], df_slopes['Gradien'])
equation = f"y = {slope:.5f}x + {intercept:.5f}"
r_squared = r_value**2

# Plot regresi linear 
plt.plot(df_slopes['Jarak (cm)'], df_slopes['Gradien'], 'o', label='Data')
plt.plot(df_slopes['Jarak (cm)'], intercept + slope * df_slopes['Jarak (cm)'], 'r', label='Regresi Linear')
plt.xlabel('Jarak (cm)')
plt.ylabel('Gradien')
plt.title("Plot Gradien terhadap Jarak")
plt.legend()
plt.text(0.55, 0.5, f"{equation}\nR²= {r_squared:.2f}", transform=plt.gca().transAxes)
plt.show()

### test menggunakan random data

In [ ]:
import pandas as pd
from scipy.stats import linregress

# Read data dari file csv 
df_test = pd.read_csv('random16.csv', sep=';', header = None)
actual_position = (-10,15) # ganti dengan titik aktual
circle_radius = 5

# Buat headers
headers = ['SimpleRx Starting', 'Time', 'Module', 'Temperature', 'Humidity']
df_test.columns = headers

# Convert kolom 'Time' menjadi tipe string
if not isinstance(df_test['Time'].iloc[0], str):
    df_test['Time'] = df_test['Time'].astype(str)

# Replace '.' dengan ':' pada kolom 'Time' 
df_test['Time'] = df_test['Time'].str.replace('.', ':', regex=False)

# Konversi waktu ke detik 'TimeInSeconds'
df_test['TimeInSeconds'] = pd.to_timedelta(df_test['Time']).dt.total_seconds()  
df_test['TimeInSeconds'] -= df_test['TimeInSeconds'].min() - 1  # kurangi dengan nilai minimum dan tambahkan 1
df_test = df_test[(df_test['TimeInSeconds']>=0)  & (df_test['TimeInSeconds']<=350)]

# Sorting data berdaasrkan sensor (A, B, C, D)
df_test = df_test.sort_values(by='TimeInSeconds')
df_test

In [ ]:
# Hitung gradien untuk tiap titik sensor 
slopes = []
points = df_test['Module'].unique()
for point in points:
    data = df_test[df_test['Module'] == point]
    slopeT, interceptT, _, _, _ = linregress(data['TimeInSeconds'], data['Temperature'])
    slopes.append(slopeT)

df_test_slopes = pd.DataFrame({'Module': points, 'Slope': slopes})
df_test_slopes = df_test_slopes.sort_values(by='Module', ascending=True)
df_test_slopes

In [ ]:
# Plot tiap titik sensor
fig, ax = plt.subplots()
for i, point in enumerate(points):
    data = df_test[df_test['Module'] == point]
    ax.scatter(data['TimeInSeconds'], data['Temperature'], label=point, s=2)
    
ax.set_xlabel('Waktu (s)')
ax.set_ylabel('Temperatur (°C)')
ax.legend()
plt.show()

In [ ]:
predicted_distance = (df_test_slopes['Slope'] - intercept) / slope
distance_pred = pd.DataFrame({'Distance': predicted_distance})

df_merged = pd.concat([df_test_slopes, distance_pred], axis=1)
df_merged

In [ ]:
import numpy as np
from matplotlib.patches import Circle

# Koordinat sensor-sensor yang digunakan dalam deteksi (koordinat sensor suhu)
sensor_coords = np.array([
    [0, 50],
    [50, 0],
    [0, -50],
    [-50, 0]
])

sensor_labels = ['A', 'B', 'C', 'D']

# Jarak antara setiap sensor dengan titik api (predicted distance) 
d1 = df_merged[df_merged['Module'] == 'A']['Distance'].values[0]
d2 = df_merged[df_merged['Module'] == 'B']['Distance'].values[0]
d3 = df_merged[df_merged['Module'] == 'C']['Distance'].values[0]
d4 = df_merged[df_merged['Module'] == 'D']['Distance'].values[0]

distances = np.array([d1, d2, d3, d4])

# Multilaterasi
def multilateration(sensor_coords, distances):
    A = 2 * (sensor_coords[1:] - sensor_coords[0])
    b = np.square(sensor_coords[1:, 0]) + np.square(sensor_coords[1:, 1]) - np.square(sensor_coords[0, 0]) - np.square(sensor_coords[0, 1]) - np.square(distances[1:]) + np.square(distances[0])
    # Menghitung posisi menggunakan metode least squares
    try:
        position = np.linalg.solve(A.T @ A, A.T @ b)
        return position
    except np.linalg.LinAlgError:
        return None

estimated_position = multilateration(sensor_coords, distances)
    
# Membuat plot
fig, ax = plt.subplots()
# Tambah lingkaran untuk posisi aktual & prediksi
patch_circle_actual = Circle((actual_position[0], actual_position[1]), circle_radius,fill=True, color='yellow', linestyle='solid', zorder=0, label='Posisi Api Aktual')
ax.add_patch(patch_circle_actual)
ax.scatter(sensor_coords[:, 0], sensor_coords[:, 1], color='dodgerblue', label='Sensor')
ax.scatter(estimated_position[0], estimated_position[1], color='red', label='Posisi Api Prediksi')

# Plot lingkaran untuk setiap jarak prediksi
for i in range(len(distances)):
    circle = Circle((sensor_coords[i][0], sensor_coords[i][1]), distances[i], fill=False, color='gray', linestyle='dashed', zorder=1)
    ax.add_patch(circle)
    ax.text(sensor_coords[i][0]+3, sensor_coords[i][1], sensor_labels[i], ha='left', va='center', color='black')

ax.text(estimated_position[0]+10, estimated_position[1]-4, f"({estimated_position[0]:.2f}, {estimated_position[1]:.2f})", ha='center', va='top', color='black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.legend()
ax.grid(True, alpha=0.2)
ax.set_title('Prediksi Posisi Titik Api')
plt.axis('equal')  
ax.set_xlim(-70, 70)
ax.set_ylim(-70, 70)
plt.show()

In [ ]:
print("Estimated position: ({:.2f}, {:.2f})".format(round(estimated_position[0], 2), round(estimated_position[1], 2)))

In [ ]:
import numpy as np

actual_x = actual_position[0]
actual_y = actual_position[1]
predicted_x = estimated_position[0]
predicted_y = estimated_position[1]

euclidean_distance = np.sqrt((actual_x - predicted_x)**2 + (actual_y - predicted_y)**2)
print("Euclidean distance:", euclidean_distance)

if euclidean_distance < 5:
    print("The predicted point is within the range of the actual point.")
else:
    print("The predicted point is outside the range of the actual point.")
